<a href="https://colab.research.google.com/github/djuanekb/forage-jpmc-swe-task-1/blob/main/backtester_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf

In [ ]:
amzn = yf.download('AMZN', '2000-01-01', '2020-01-01')


[*********************100%%**********************]  1 of 1 completed


In [ ]:
def sma(array, period):
  """ simple moving average"""
  return array.rolling(period).mean()

In [ ]:
sma14 = sma(amzn['Close'], 14)
sma200 =sma(amzn['Close'], 200)
print(sma14, sma200)

Date
2000-01-03          NaN
2000-01-04          NaN
2000-01-05          NaN
2000-01-06          NaN
2000-01-07          NaN
                ...    
2019-12-24    88.413143
2019-12-26    88.871322
2019-12-27    89.293464
2019-12-30    89.641250
2019-12-31    90.029213
Name: Close, Length: 5031, dtype: float64 Date
2000-01-03          NaN
2000-01-04          NaN
2000-01-05          NaN
2000-01-06          NaN
2000-01-07          NaN
                ...    
2019-12-24    91.192535
2019-12-26    91.237025
2019-12-27    91.282920
2019-12-30    91.316552
2019-12-31    91.342975
Name: Close, Length: 5031, dtype: float64


In [ ]:
#simple strategy to buy when shorter term crosses above lower term
#sell when short-term sma crosses below longer term

def crossover(array1, array2):
  """ buy signal"""
  return array1 > array2

def crossunder(array1, array2):
  """ sell signal"""
  return array1 < array2

enter_rules = crossover(sma14, sma200)
exit_rules = crossunder(sma14, sma200)

print(enter_rules[enter_rules.index == '2000-10-13'])

Date
2000-10-13    False
Name: Close, dtype: bool


In [ ]:
#tracking ongoing

def marketposition_generator(dataset, enter_rules, exit_rules):
  """
    Takes three arguments
    dataset: amzn datase
    enter/exit_rules: marketpostion
  """

  dataset['enter_rules'] = enter_rules
  dataset['exit_rules'] = exit_rules


  status = 0
  #a switch that: boolean for marketposition
  mp = []
  # (1) when entering
  # (0) when exiting

  for (i, j) in zip(enter_rules, exit_rules):
    if status == 0:
      if i == 1 and j != -1:
        status = 1
    else:
      if j == -1:
        status = 0
    mp.append(status)

  #trades start one day later
  dataset['mp'] = mp
  dataset['mp'] = dataset['mp'].shift(1)
  dataset.iloc[0,2] = 0

  return dataset['mp']

In [ ]:
#Defining backtest function
COST = 0.50                 #every operation costs 0.50 (0.25 to buy and sell)
INSTRUMENT = 1              #tested on Amazon stock
OPERATION_MONEY = 10000     #intitial capital
DIRECTION = 'long'          #testing for long trades
ORDER_TYPE = 'market'       #strategy processes market orders
ENTER_LEVEL = amzn['Open']  #entry price is the open price

In [ ]:
def apply_trading_system(dataset, direction, order_type, enter_level, enter_rules, exit_rules):

    #Adding the two boolean series and market position funcntion
    dataset['enter_rules'] = enter_rules.apply(lambda x: 1 if x == True else 0)
    dataset['exit_rules'] = exit_rules.apply(lambda x: -1 if x == True else 0)
    dataset['mp'] = marketposition_generator(dataset['enter_rules'], dataset['exit_rules'])

    #We put in market orders (buying quantity) if the sigal has changed on the previous day
    if order_type == "market":
        dataset['entry_price'] = np.where((dataset.mp.shift(1) == 0) &
                                             (dataset.mp == 1), dataset.Open.shift(1), np.nan)\
        #Number of shares we buy as a ratio of Capital and entry_price
        if INSTRUMENT == 1:
            dataset['number_of_stocks'] = np.where((dataset.mp.shift(1) == 0) &
                                                     (dataset.mp == 1), OPERATION_MONEY / dataset.Open, np.nan)

    #Forward propagate the value of the entry price
    dataset['entry_price'] = dataset['entry_price'].fillna(method='ffill')

    #Forward propagate value of stocks as whole numbers
    if INSTRUMENT == 1:
        dataset['number_of_stocks'] = dataset['number_of_stocks']\
                                        .apply(lambda x: round(x, 0)).fillna(method='ffill')

    #associate the label 'entry' to 'events_in' everytime mp moves from 0 to 1
    dataset['events_in'] = np.where((dataset.mp == 1) & (dataset.mp.shift(1) == 0), 'entry', '')

    #Defining long trades
    if direction == 'long':
        if INSTRUMENT == 1:
            #
            dataset['open_operations'] = (dataset.Close - dataset.entry_price) * dataset.number_of_stocks
            dataset['open_operations'] = np.where((dataset.mp == 1) & (dataset.mp.shift(-1) == 0),
                                                    (dataset.Open.shift(-1) - dataset.entry_price) * dataset.number_of_stocks - 2 * COSTS,
                                                     dataset.open_operations)
    else:
        if INSTRUMENT == 1:
            dataset['open_operations'] = (dataset.entry_price - dataset.Close) * dataset.number_of_stocks
            dataset['open_operations'] = np.where((dataset.mp == 1) & (dataset.mp.shift(-1) == 0),
                                                    (dataset.entry_price - dataset.Open.shift(-1)) * dataset.number_of_stocks - 2 * COSTS,
                                                     dataset.open_operations)

    dataset['open_operations'] = np.where(dataset.mp == 1, dataset.open_operations, 0)
    dataset['events_out'] = np.where((dataset.mp == 1) & (dataset.exit_rules == -1), 'exit', '')
    dataset['operations'] = np.where((dataset.exit_rules == -1) &
                                       (dataset.mp == 1), dataset.open_operations, np.nan)
    dataset['closed_equity'] = dataset.operations.fillna(0).cumsum()
    dataset['open_equity'] = dataset.closed_equity + dataset.open_operations - dataset.operations.fillna(0)

    dataset.to_csv('trading_system_export.csv')

    return dataset